In [1]:
# importacao de bibliotecas
import pandas as pd
from unidecode import unidecode

%load_ext sql

In [2]:
# Importacao da biblioteca sqlalchemy
from sqlalchemy import create_engine

In [3]:
## Conexão com a base de dados postgree

import pyodbc as podbc

cnxn = podbc.connect("Driver={SQL Server Native Client 10.0};Server=hspmins17;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")

In [3]:
## Conexão com a base de dados postgre

import pyodbc as podbc

cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins20;Database=Isolamento_Versao_2;uid=sa;pwd=Hspm@banco123")

In [35]:
## Acesso a base de dados

sql = "SELECT * FROM [Isolamento_Versao_2].[dbo].[Laboratorio] WHERE     (Prontuario IS NULL)"
data_sql_server = pd.read_sql(sql,cnxn)

In [36]:
data_sql_server.shape


(173, 12)

In [7]:
data_sql_server.NomePaciente

0       DAMARIS MOURÃO BERTOLINO
1                 KIYOE ISHIMOTO
2      VIVIANE JENNIFER DA SILVA
3       ALZIRA LINO GARCIA SILVA
4            ERICK SILVA TARGINO
                 ...            
347      WALDIR PRIMO FIORAVANTI
348              WALTER DEL NERO
349         YEDA APPARECIDA DIAS
350         YEDA APPARECIDA DIAS
351         YEDA APPARECIDA DIAS
Name: NomePaciente, Length: 352, dtype: object

In [32]:
# Conectar com a base de desenvolvimento do SGH e buscar os pacientes listados no dataframe data.
import psycopg2
con = psycopg2.connect(host='pg019dbsrv.prodam', database='hm0615_hspm', port ="5432",user='hm0615_hspm', password='pwd_hm0615_hspm')


In [9]:
sql = "SELECT * FROM agh.aip_pacientes limit 100;"
data_postgre = pd.read_sql(sql,con)

In [10]:
data_postgre.head()

,codigo,cct_codigo_cadastro,ser_matricula_cadastro,ser_vin_codigo_cadastro,nome,nome_mae,dt_nascimento,dt_identificacao,cct_codigo_recadastro,cdd_codigo,...,cd_rf_matricula,orgao,ddd_fone_comercial,fone_comercial,email,nome_titular,validade_termino,situacao_vinculo,causa_termino,grau_dep_vinculo
0,28,1,9999999,955,ANA EUGENIA DOS SANTOS,MAE DA ANA,1911-03-25,2019-06-29 11:27:44.005,NaN,9668,...,None,None,None,None,None,None,NaT,None,None,None
1,30,1,9999999,955,VITORIA PEIXOTO RUIZ,MARIA BISAO,1926-09-23,2019-06-29 12:17:33.678,NaN,9668,...,None,None,None,None,None,None,NaT,None,None,None
2,31,1,9999999,955,MARIANA DE ALMEIDA BREVIGLIERI,SILVANA SALLES ALMEIDA BREVIGLIERI,1985-09-19,2019-06-29 12:25:06.630,NaN,9668,...,281970,MUNICIPE,None,None,None,None,NaT,None,None,None
3,32,1,9999999,955,APARECIDA ALEXANDRIA DE FARIA ASSIS,MARIA CANDIDA O DE FARIA,1965-03-13,2019-06-29 16:02:17.558,NaN,9668,...,None,None,None,None,None,None,NaT,None,None,None
4,34,1,9999999,955,ARIANE SANTOS DE OLIVEIRA,MARISA APARECIDA DOS SANTOS,1999-11-30,2019-06-29 17:16:19.496,NaN,9668,...,None,None,None,None,None,None,NaT,None,None,None


In [11]:
data_postgre.prontuario

0         2642
1        44354
2       765628
3       135335
4       115899
        ...   
95      240861
96    12081568
97    10896983
98    10169118
99    11436920
Name: prontuario, Length: 100, dtype: int64

In [12]:
# Primeiro, encontrar todos os nomes de pacientes e a data de nascimento para construção da query
# que fará o looping no banco de dadsos do SGH

sql = "SELECT NomePaciente, DataNascimento FROM [Isolamento_Versao_2].[dbo].[Laboratorio] WHERE     (Prontuario IS NULL)  group by NomePaciente, DataNascimento"
data_pacientes_laboratorio = pd.read_sql(sql,cnxn)


In [13]:
data_pacientes_laboratorio.shape

(238, 2)

In [27]:
data_pacientes_laboratorio.head(10)

,NomePaciente,DataNascimento
0,ADEMIR FRANCO DA SILVA,1964-08-28
1,ADRIANA TREZENO TENORIO,1985-01-13
2,ALCINEIA IDALGO DE ALMEIDA,1975-07-30
3,ALEXANDRE DA SILVA CARDOSO,1971-08-28
4,ALEXIA LEANDRO MENDES,1970-05-22
5,ALICE YUEN,2021-02-23
6,ALZIRA LINO GARCIA SILVA,1977-08-20
7,AMANDA CRISTINA LEAO,1980-12-19
8,ANA BEATRIZ CARDOSO SILVA,1997-11-22
9,ANA LUCIA SCUCCUGLIA,1967-06-07


In [33]:


for index, row in data_pacientes_laboratorio.iterrows():
  
   
  
    
    sql1 = "SELECT prontuario FROM agh.aip_pacientes where nome like '" + unidecode(row[0]) + "%' and dt_nascimento = '" + str(row[1]) + "';"
    dp2 = pd.read_sql(sql1,con)
    
    if not dp2.empty:
        print(row[0])
        print('passei')
        print(str( dp2.prontuario[0]))
        sql2 = "UPDATE [Isolamento_Versao_2].[dbo].[Laboratorio] SET Prontuario = " +str( dp2.prontuario[0]) + " WHERE NomePaciente = '" + unidecode(row[0]) + "';"
        cnxn.execute(sql2)
  

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ADRIANA TREZENO TENORIO
passei
11382587
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ALCINEIA IDALGO DE ALMEIDA
passei
5009949
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ALEXANDRE DA SILVA CARDOSO
passei
10934255
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ANEZIA LACERDA DE SOUZA SILVA
passei
1960889
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ANNA QUINTANA


LIS CONCEICAO NOVAES
passei
11261393
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
LUIZ CARLOS MOREIRA DOS SANTOS
passei
3360815
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
LUIZA OLIVEIRA DE ANDRADE FRANCISCO
passei
11696747
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
MARCELINA MOTTA E SILVA CUNHA
passei
2141554
<cla

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
YEDA APPARECIDA DIAS
passei
120535


In [34]:
cnxn.commit() 




In [ ]:
'''
for x in data_sql_server:
    print(x)
    print(type(x))
    y = str(x) + "%"
    sql = "SELECT prontuario FROM agh.aip_pacientes where prontuario::varchar(255) like '" + "1160946%"  + "';" 
    dp = pd.read_sql(sql,con)
    if(dp.empty != True):
        print(type(dp.prontuario[0]))
        z = dp.prontuario[0].astype(str)
        print(z)
'''  

In [ ]:
data_sql_server.head()

In [ ]:
data_sql_server.info()

In [ ]:
data_sql_server.describe()

In [ ]:
# descobrir os outliers de datas da base de dados do Sistema Isolamento.
cnxn = podbc.connect("Driver={SQL Server Native Client 10.0};Server=hspmins17;Database=Isolamento;uid=hspmApp;pwd=SoundG@rden=1")
sql = "SELECT [rh],[nome],[dt_nasc],[sexo],[obito],[dtobito] FROM [Isolamento].[dbo].[Paciente]"
data_paciente = pd.read_sql(sql,cnxn)

In [ ]:
data_paciente.head()

In [ ]:
# Python3 code to calculate age in years 

from datetime import date 

def calculateAge(birthDate): 
    today = date.today() 
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day)) 

    return age 

# Driver code 
print(calculateAge(date(1997, 2, 3)), "years") 


In [ ]:
#Adicionar em um DataFrame todos as idades e criar box plot para enxergar os outliers
rows_list = []

for x in data_paciente.dt_nasc:
    print(calculateAge(x), "years") 
    rows_list.append(calculateAge(x))
    
df = pd.DataFrame(rows_list)    

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1178 entries, 0 to 1177
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1178 non-null   int64
dtypes: int64(1)
memory usage: 9.3 KB


In [30]:
import seaborn as sns

sns.set(style="whitegrid")
tips = sns.load_dataset(df)
ax = sns.boxplot(x=tips["dt_nasc"])

InvalidURL: URL can't contain control characters. '/mwaskom/seaborn-data/master/       0\n0     82\n1     92\n2     65\n3     84\n4     74\n...   ..\n1173  51\n1174  45\n1175  57\n1176  74\n1177  70\n\n[1178 rows x 1 columns].csv' (found at least ' ')

In [130]:
pip install unidecode


     -------------------------------------- 235.9/235.9 kB 3.6 MB/s eta 0:00:00

